# Data Preprocessing

### Libraries to import:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# plt.style.use('fivethirtyeight')
np.random.seed(365)
import pickle

from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

import folium

#### Reading in dataset for January for yellow cabs

In [2]:
jan_df = pd.read_csv('./data/2020/yellow_2020-01.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
output = assign_borough(jan_df, borough_info)

In [15]:
output.Borough.value_counts()

Manhattan        5846105
Queens            425439
Brooklyn           71164
Bronx              14073
Staten Island        354
Name: Borough, dtype: int64

In [3]:
jan_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


In [4]:
borough_info = pd.read_csv('./data/taxi_zones.csv')
borough_info.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


# Data Cleaning

We will clean our monthly data so that we are left with correct datetime object as index and drop any values that are inputted incorrectly.

In [5]:
jan_df['tpep_pickup_datetime'].sort_values()

782435     2003-01-01 00:07:17
5086498    2008-12-31 23:02:40
1447622    2008-12-31 23:02:50
5086894    2008-12-31 23:03:44
3073519    2008-12-31 23:03:48
                  ...         
4269480    2020-07-10 11:34:11
4282277    2020-07-31 18:50:41
275044     2021-01-02 00:22:00
275045     2021-01-02 00:44:08
275046     2021-01-02 01:12:10
Name: tpep_pickup_datetime, Length: 6405008, dtype: object

For some reason, our dataset which should only have January 2020 data but starts from 2003, 2008, and ends at 2021 and even months which make no sense. So I will have to drop those rows that are not in 2020 of january. 

In [6]:
def assign_borough(df, borough_info):
    conditions = [
        df['PULocationID'].eq(2)|df['PULocationID'].eq(7)|df['PULocationID'].eq(8)|df['PULocationID'].eq(9)|df['PULocationID'].eq(10)|df['PULocationID'].eq(15)|df['PULocationID'].eq(16)|df['PULocationID'].eq(19)|df['PULocationID'].eq(27)|df['PULocationID'].eq(28)|df['PULocationID'].eq(30)|df['PULocationID'].eq(38)|df['PULocationID'].eq(53)|df['PULocationID'].eq(56)|df['PULocationID'].eq(57)|df['PULocationID'].eq(64)|df['PULocationID'].eq(70)|df['PULocationID'].eq(73)|df['PULocationID'].eq(82)|df['PULocationID'].eq(83)|df['PULocationID'].eq(86)|df['PULocationID'].eq(92)|df['PULocationID'].eq(93)|df['PULocationID'].eq(95)|df['PULocationID'].eq(96)|df['PULocationID'].eq(98)|df['PULocationID'].eq(101)|df['PULocationID'].eq(102)|df['PULocationID'].eq(117)|df['PULocationID'].eq(121)|df['PULocationID'].eq(122)|df['PULocationID'].eq(124)|df['PULocationID'].eq(129)|df['PULocationID'].eq(130)|df['PULocationID'].eq(131)|df['PULocationID'].eq(132)|df['PULocationID'].eq(134)|df['PULocationID'].eq(135)|df['PULocationID'].eq(138)|df['PULocationID'].eq(139)|df['PULocationID'].eq(145)|df['PULocationID'].eq(146)|df['PULocationID'].eq(157)|df['PULocationID'].eq(160)|df['PULocationID'].eq(171)|df['PULocationID'].eq(173)|df['PULocationID'].eq(175)|df['PULocationID'].eq(179)|df['PULocationID'].eq(180)|df['PULocationID'].eq(191)|df['PULocationID'].eq(192)|df['PULocationID'].eq(193)|df['PULocationID'].eq(196)|df['PULocationID'].eq(197)|df['PULocationID'].eq(198)|df['PULocationID'].eq(201)|df['PULocationID'].eq(203)|df['PULocationID'].eq(205)|df['PULocationID'].eq(207)|df['PULocationID'].eq(215)|df['PULocationID'].eq(216)|df['PULocationID'].eq(218)|df['PULocationID'].eq(219)|df['PULocationID'].eq(223)|df['PULocationID'].eq(226)|df['PULocationID'].eq(252)|df['PULocationID'].eq(253)|df['PULocationID'].eq(258)|df['PULocationID'].eq(260),
        df['PULocationID'].eq(3)|df['PULocationID'].eq(18)|df['PULocationID'].eq(20)|df['PULocationID'].eq(31)|df['PULocationID'].eq(32)|df['PULocationID'].eq(46)|df['PULocationID'].eq(47)|df['PULocationID'].eq(51)|df['PULocationID'].eq(58)|df['PULocationID'].eq(59)|df['PULocationID'].eq(60)|df['PULocationID'].eq(69)|df['PULocationID'].eq(78)|df['PULocationID'].eq(81)|df['PULocationID'].eq(94)|df['PULocationID'].eq(119)|df['PULocationID'].eq(126)|df['PULocationID'].eq(136)|df['PULocationID'].eq(147)|df['PULocationID'].eq(159)|df['PULocationID'].eq(167)|df['PULocationID'].eq(168)|df['PULocationID'].eq(169)|df['PULocationID'].eq(174)|df['PULocationID'].eq(182)|df['PULocationID'].eq(183)|df['PULocationID'].eq(184)|df['PULocationID'].eq(185)|df['PULocationID'].eq(199)|df['PULocationID'].eq(200)|df['PULocationID'].eq(208)|df['PULocationID'].eq(212)|df['PULocationID'].eq(213)|df['PULocationID'].eq(220)|df['PULocationID'].eq(235)|df['PULocationID'].eq(240)|df['PULocationID'].eq(241)|df['PULocationID'].eq(242)|df['PULocationID'].eq(247)|df['PULocationID'].eq(248)|df['PULocationID'].eq(250)|df['PULocationID'].eq(254)|df['PULocationID'].eq(259),
        df['PULocationID'].eq(4)|df['PULocationID'].eq(12)|df['PULocationID'].eq(13)|df['PULocationID'].eq(24)|df['PULocationID'].eq(41)|df['PULocationID'].eq(42)|df['PULocationID'].eq(43)|df['PULocationID'].eq(45)|df['PULocationID'].eq(48)|df['PULocationID'].eq(50)|df['PULocationID'].eq(68)|df['PULocationID'].eq(74)|df['PULocationID'].eq(75)|df['PULocationID'].eq(79)|df['PULocationID'].eq(87)|df['PULocationID'].eq(88)|df['PULocationID'].eq(90)|df['PULocationID'].eq(100)|df['PULocationID'].eq(103)|df['PULocationID'].eq(104)|df['PULocationID'].eq(105)|df['PULocationID'].eq(107)|df['PULocationID'].eq(113)|df['PULocationID'].eq(114)|df['PULocationID'].eq(116)|df['PULocationID'].eq(120)|df['PULocationID'].eq(125)|df['PULocationID'].eq(127)|df['PULocationID'].eq(128)|df['PULocationID'].eq(137)|df['PULocationID'].eq(140)|df['PULocationID'].eq(141)|df['PULocationID'].eq(142)|df['PULocationID'].eq(143)|df['PULocationID'].eq(144)|df['PULocationID'].eq(148)|df['PULocationID'].eq(151)|df['PULocationID'].eq(152)|df['PULocationID'].eq(153)|df['PULocationID'].eq(158)|df['PULocationID'].eq(161)|df['PULocationID'].eq(162)|df['PULocationID'].eq(163)|df['PULocationID'].eq(164)|df['PULocationID'].eq(166)|df['PULocationID'].eq(170)|df['PULocationID'].eq(186)|df['PULocationID'].eq(194)|df['PULocationID'].eq(202)|df['PULocationID'].eq(209)|df['PULocationID'].eq(211)|df['PULocationID'].eq(224)|df['PULocationID'].eq(229)|df['PULocationID'].eq(230)|df['PULocationID'].eq(231)|df['PULocationID'].eq(232)|df['PULocationID'].eq(233)|df['PULocationID'].eq(234)|df['PULocationID'].eq(236)|df['PULocationID'].eq(237)|df['PULocationID'].eq(238)|df['PULocationID'].eq(239)|df['PULocationID'].eq(243)|df['PULocationID'].eq(244)|df['PULocationID'].eq(246)|df['PULocationID'].eq(249)|df['PULocationID'].eq(261)|df['PULocationID'].eq(262)|df['PULocationID'].eq(263),
        df['PULocationID'].eq(5)|df['PULocationID'].eq(6)|df['PULocationID'].eq(23)|df['PULocationID'].eq(44)|df['PULocationID'].eq(84)|df['PULocationID'].eq(99)|df['PULocationID'].eq(109)|df['PULocationID'].eq(110)|df['PULocationID'].eq(115)|df['PULocationID'].eq(118)|df['PULocationID'].eq(156)|df['PULocationID'].eq(172)|df['PULocationID'].eq(176)|df['PULocationID'].eq(187)|df['PULocationID'].eq(204)|df['PULocationID'].eq(206)|df['PULocationID'].eq(214)|df['PULocationID'].eq(221)|df['PULocationID'].eq(245)|df['PULocationID'].eq(251),
        df['PULocationID'].eq(11)|df['PULocationID'].eq(14)|df['PULocationID'].eq(17)|df['PULocationID'].eq(21)|df['PULocationID'].eq(22)|df['PULocationID'].eq(25)|df['PULocationID'].eq(26)|df['PULocationID'].eq(29)|df['PULocationID'].eq(33)|df['PULocationID'].eq(34)|df['PULocationID'].eq(35)|df['PULocationID'].eq(36)|df['PULocationID'].eq(37)|df['PULocationID'].eq(39)|df['PULocationID'].eq(40)|df['PULocationID'].eq(49)|df['PULocationID'].eq(52)|df['PULocationID'].eq(54)|df['PULocationID'].eq(55)|df['PULocationID'].eq(61)|df['PULocationID'].eq(62)|df['PULocationID'].eq(63)|df['PULocationID'].eq(65)|df['PULocationID'].eq(66)|df['PULocationID'].eq(67)|df['PULocationID'].eq(71)|df['PULocationID'].eq(72)|df['PULocationID'].eq(76)|df['PULocationID'].eq(77)|df['PULocationID'].eq(80)|df['PULocationID'].eq(85)|df['PULocationID'].eq(89)|df['PULocationID'].eq(91)|df['PULocationID'].eq(97)|df['PULocationID'].eq(106)|df['PULocationID'].eq(108)|df['PULocationID'].eq(111)|df['PULocationID'].eq(112)|df['PULocationID'].eq(123)|df['PULocationID'].eq(133)|df['PULocationID'].eq(149)|df['PULocationID'].eq(150)|df['PULocationID'].eq(154)|df['PULocationID'].eq(155)|df['PULocationID'].eq(165)|df['PULocationID'].eq(177)|df['PULocationID'].eq(178)|df['PULocationID'].eq(181)|df['PULocationID'].eq(188)|df['PULocationID'].eq(189)|df['PULocationID'].eq(190)|df['PULocationID'].eq(195)|df['PULocationID'].eq(210)|df['PULocationID'].eq(217)|df['PULocationID'].eq(222)|df['PULocationID'].eq(225)|df['PULocationID'].eq(227)|df['PULocationID'].eq(228)|df['PULocationID'].eq(255)|df['PULocationID'].eq(256)|df['PULocationID'].eq(257),
        df['PULocationID'].eq(1)|df['PULocationID'].eq(264)|df['PULocationID'].eq(265)
    ]
    choices = [
        'Queens',
        'Bronx',
        'Manhattan',
        'Staten Island',
        'Brooklyn',
        np.nan
    ]
    df['Borough'] = np.select(conditions, choices, df['PULocationID'])
    df = df[df['Borough']!='nan']
    return df

    

14073

In [10]:
def clean_pickup_time(dataframe, borough_info, year, month):
    # Assign Boroguh
    time_df = assign_borough(dataframe, borough_info)
    # Subset the pickup time, rename column, and sort
    time_df =dataframe[['tpep_pickup_datetime', 'PULocationID', 'tip_amount', 'total_amount', 'trip_distance', 'Borough']]
    time_df = time_df.rename(columns={"tpep_pickup_datetime": "pickup_time"})
    time_df = time_df.sort_values(by='pickup_time')
    time_df['pickup_time'] = time_df['pickup_time'].map(lambda x: np.nan if (str(x[0:4])!=year or str(x[5:7]) != month ) else x)
    print('missing values before cleaning:', time_df.isna().sum())
    time_df.dropna(subset=['pickup_time'], inplace=True)
    print('missing values after cleaning:', time_df.isna().sum())
    #pickle data
    pickle.dump(time_df, open(f'./data/pickled/alldata_{year}_{month}.pkl','wb'))

    #time_df.to_csv(f'./data/cleaned_data/sorted_{year}_{month}')

    # Assigning PickuplocationID to one of the five buroughs
    
    # add count column 
    time_df['count'] = time_df['pickup_time'].map(lambda x:1)
    # convert to datetime object
    time_df['pickup_time'] = time_df['pickup_time'].map(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S'))
    time_df = time_df[['pickup_time', 'count']]

    # setting index
    time_df = time_df.set_index(pd.DatetimeIndex(time_df['pickup_time']), drop=True)

    # Resample by hourly
    time_df = time_df.resample('H').sum()

    print('data cleaning complete!')
    # Save csv file to cleaned_data folder
    
    #pickle data before indexing
    pickle.dump(time_df, open(f'./data/pickled/ts_{year}_{month}.pkl','wb'))

    # time_df.to_csv(f'./data/time_series_data/ts_{year}_{month}', date_format='%Y-%m-%d %H:%M:%S')
    return time_df

In [11]:

borough_info = pd.read_csv('./data/taxi_zones.csv')


In [12]:
cleaned_jan_df = clean_pickup_time(jan_df, borough_info, '2020', '01')

missing values before cleaning: pickup_time      212
PULocationID       0
tip_amount         0
total_amount       0
trip_distance      0
Borough            0
dtype: int64
missing values after cleaning: pickup_time      0
PULocationID     0
tip_amount       0
total_amount     0
trip_distance    0
Borough          0
dtype: int64
data cleaning complete!


In [ ]:
cleaned_jan_df

In [ ]:
import pickle
# pickle.dump(output, open('./data/pickled/data.pkl','wb'))
